In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr


In [2]:
df = pd.read_csv('../../Data/data_GLDAS/compiled_canada_soil_moisture.csv')

In [3]:
df.columns

Index(['time', 'lat', 'lon', 'SoilMoi0_10cm_inst', 'SoilMoi10_40cm_inst',
       'SoilMoi40_100cm_inst', 'SoilMoi100_200cm_inst'],
      dtype='object')

In [4]:
df = df.drop(['SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilMoi100_200cm_inst'], axis = 1)
# rename the column 'SoilMoi0_10cm_inst' to 'waterstorage'
df = df.rename(columns={'SoilMoi0_10cm_inst': 'waterstorage'})
# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']))
# gdf = gdf.set_crs(epsg=4326, inplace=True)

# gdf.to_parquet('../Data/data_GLDAS/gdf_compiled_canada_soil_moisture.parquet', index=False)

#create a simple df with 4 entries which build a coordinate grid with 0.25° resolution
# df_test = pd.DataFrame(columns=['lon', 'lat', 'waterstorage'])
# lon = []
# lat = []
# for i in range(4):
#     for j in range(4):
#         lon_ = 100.125 + i * 0.25
#         lat_ = 10.125 + j * 0.25
#         lon.append(lon_)
#         lat.append(lat_)
# # list_lon
# df_test = pd.DataFrame({
#     'lon': lon,
#     'lat': lat,
#     'waterstorage': np.random.rand(16),
#     'time': pd.datetime(2023, 1, 1) 
# })

# df_test

In [5]:

# --- Configuration ---
# IMPORTANT: Replace 'your_large_file.csv' with the actual path to your file


output_csv_file = '../../output/downsampled_1deg_water.csv'

# Define the input and output grid resolutions
lat_res_in = 0.25
lon_res_in = 0.25
lat_res_out = 1.0
lon_res_out = 1.0
# --- End Configuration ---

print(f"Starting script...")
print(f"Output file: {output_csv_file}")
print(f"Input resolution: {lat_res_in}° x {lon_res_in}°")
print(f"Output resolution: {lat_res_out}° x {lon_res_out}°")

# --- Step 1: Read the CSV data using Pandas ---
print(f"\nReading input file...")
# For very large files (like 600MB), this might consume a lot of memory.
# If you encounter MemoryError, consider:
# 1. Using Dask DataFrames: `import dask.dataframe as dd; df = dd.read_csv(...)`
#    which reads and processes data in chunks.
# 2. Using Pandas chunking: Iterate through chunks of the CSV
#    `chunks = pd.read_csv(input_csv_file, chunksize=100000)` and process chunk by chunk.
# This example assumes the file fits into memory for simplicity.
try:
    df = df
    # Optional: Convert 'time' column to datetime objects if it's not already
    # df['time'] = pd.to_datetime(df['time'])
    print("CSV read complete.")
    print("DataFrame head:")
    print(df.head())
except MemoryError:
    print("\n---------------------------------------------------------")
    print("MemoryError: The file is too large to read directly into memory with Pandas.")
    print("Consider using Dask DataFrames (`dask.dataframe`) or ")
    print("reading the file in chunks with `pd.read_csv(..., chunksize=...)`.")
    print("---------------------------------------------------------")
    exit() # Exit the script if memory error occurs
except FileNotFoundError:
    print(f"\nError: Input file not found   ")
    print("Please ensure the file path is correct.")
    exit()

# --- Step 2: Convert Pandas DataFrame to Xarray Dataset ---
# Xarray works best with multi-dimensional data. We need to set
# 'time', 'lat', 'lon' as index dimensions for the conversion.
print("\nConverting DataFrame to Xarray Dataset...")
try:
    df = df.set_index(['time', 'lat', 'lon'])
    ds = df.to_xarray()
    # Ensure latitude is decreasing if needed (common in climate data)
    # if ds['lat'][0] < ds['lat'][-1]:
    #     ds = ds.reindex(lat=list(reversed(ds['lat'])))
    print("Xarray Dataset created successfully:")
    print(ds)
except KeyError as e:
     print(f"\nError: Column '{e}' not found in CSV. Needed for setting index.")
     print("Please ensure your CSV has 'time', 'lat', 'lon' columns.")
     exit()
except Exception as e:
    print(f"\nAn error occurred during DataFrame to Xarray conversion: {e}")
    print("This might happen if there are duplicate time/lat/lon combinations.")
    # Optional: Add code here to check for and handle duplicates if necessary
    # print("Checking for duplicate index entries...")
    # duplicates = df.index.duplicated(keep=False)
    # print(f"Found {duplicates.sum()} duplicate index entries.")
    # print("Example duplicates:")
    # print(df[duplicates].head())
    exit()


# --- Step 3: Perform the spatial downsampling (coarsening) ---
# Calculate the window size needed to go from 0.25° to 1°
# Ensure integer division or handle potential floating point issues if resolutions were different
lat_window = int(lat_res_out / lat_res_in)
lon_window = int(lon_res_out / lon_res_in)

if lat_window <= 0 or lon_window <= 0:
    print("\nError: Output resolution must be larger than input resolution for coarsening.")
    exit()

print(f"\nCoarsening data with latitude window={lat_window}, longitude window={lon_window}...")

# Use the coarsen method. It selects blocks of size {lat: lat_window, lon: lon_window}
# and applies the function (in this case, mean) to each block.
# 'boundary="pad"' handles edges where the dimensions might not be perfectly
# divisible by the window size (shouldn't be an issue for 0.25 -> 1).
try:
    ds_coarse = ds.coarsen(lat=lat_window, lon=lon_window, boundary='pad').mean()
    print("Coarsening complete.")
    print("Downsampled Dataset:")
    print(ds_coarse)
except Exception as e:
    print(f"\nAn error occurred during coarsening: {e}")
    print("Check if latitude/longitude coordinates are regularly spaced.")
    exit()

# Note on coordinates: The coordinates of the coarse grid ('lat', 'lon')
# usually represent the center/start of the *first* cell in each window by default.
# If you need coordinates representing the center of the new 1°x1° cell,
# you might need to calculate and assign them manually after coarsening.
# Example:
# new_lat = ds_coarse['lat'] + (lat_res_out / 2.0) - (lat_res_in / 2.0)
# new_lon = ds_coarse['lon'] + (lon_res_out / 2.0) - (lon_res_in / 2.0)
# ds_coarse = ds_coarse.assign_coords(lat=new_lat, lon=new_lon)


# --- Step 4: Convert back to DataFrame and save to CSV ---
print("\nConverting downsampled Dataset back to DataFrame...")
# Use .reset_index() to turn the coordinates ('time', 'lat', 'lon')
# back into columns for the CSV output.
df_coarse = ds_coarse.to_dataframe().reset_index()

#drop the nan rows
df_coarse = df_coarse.dropna(subset=['waterstorage'])


print(f"Saving downsampled data to {output_csv_file}...")
try:
    df_coarse.to_csv(output_csv_file, index=False, float_format='%.5f') # Control float precision
    df_coarse.to_parquet('../../output/downsampled_1deg_water.parquet', index=False)
    # print(df_coarse)
    print("Downsampled data saved successfully.")
except Exception as e:
    print(f"\nAn error occurred while saving the CSV: {e}")

print("\nScript finished.")

Starting script...
Output file: ../../output/downsampled_1deg_water.csv
Input resolution: 0.25° x 0.25°
Output resolution: 1.0° x 1.0°

Reading input file...
CSV read complete.
DataFrame head:
         time     lat      lon  waterstorage
0  2000-01-01  82.875  -67.625     32.961132
1  2000-01-01  64.875 -103.875     26.860878
2  2000-01-01  64.875 -104.125     26.629623
3  2000-01-01  64.875 -104.375     26.186129
4  2000-01-01  64.875 -104.625     25.803741

Converting DataFrame to Xarray Dataset...
Xarray Dataset created successfully:
<xarray.Dataset> Size: 144MB
Dimensions:       (time: 300, lat: 168, lon: 356)
Coordinates:
  * time          (time) object 2kB '2000-01-01' '2000-02-01' ... '2024-12-01'
  * lat           (lat) float64 1kB 41.12 41.38 41.62 ... 82.38 82.62 82.88
  * lon           (lon) float64 3kB -140.9 -140.6 -140.4 ... -52.38 -52.12
Data variables:
    waterstorage  (time, lat, lon) float64 144MB nan nan nan nan ... nan nan nan

Coarsening data with latitude window=